In [1]:
import numpy as np
import osqp
import matplotlib.pyplot as plt
from scipy import sparse
import control as ct
from casadi import *
import dpilqr
import itertools
import cvxpy as cp

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from solvers.util import (
    compute_pairwise_distance,
    compute_pairwise_distance_nd_Sym,
    define_inter_graph_threshold,
    distance_to_goal,
    split_graph, 
    generate_f,
    generate_f_human_drone,
    objective,
    generate_min_max_input,
    generate_min_max_state
)

In [4]:
def linear_kinodynamics(dt,n_agent):
    #Decision vector is a = [a_x, a_y, a_z]
    #State vector is X = [p_x, p_y, p_z, v_x, v_y, v_z]
    #Discretization time step is dt
    A_tot = sparse.lil_matrix((6*n_agent, 6*n_agent))
    B_tot = sparse.lil_matrix((6*n_agent, 3*n_agent))
    A = sparse.csc_matrix([[1, 0, 0, dt, 0, 0],
                           [0, 1, 0, 0 , dt ,0],\
                           [0, 0, 1, 0, 0 , dt],\
                           [0, 0, 0, 1, 0 ,0],\
                           [0, 0, 0, 0, 1 ,0],\
                           [0, 0, 0, 0, 0, 1]])
    B = sparse.csc_matrix([[dt**2/2, 0, 0],\
                           [0, dt**2/2, 0],\
                           [0, 0, dt**2/2],\
                           [dt, 0, 0 ],\
                           [0, dt , 0],\
                           [0, 0, dt]])

    for i in range(n_agent):
        A_tot[i*6:(i+1)*6,i*6:(i+1)*6] = A
        B_tot[i*6:(i+1)*6,i*3:(i+1)*3] = B
        
    
    return A_tot, B_tot

In [7]:
from solvers import util
from multiprocessing import Process, Pipe
import cvxpy as cp

### Consensus MPC

In [15]:
# class AdmmMPC:
#     def __init__(self, nx, nu, x0, xr, T, Q, R, Qf, N, Ad, Bd, radius = 0.3, MAX_ITER=10):
#         nx = nx #Total No. of states
#         nu = nu #Total No. of inputs
#         xr = xr #Final cond.
#         x0 = x0 #Initial cond.
#         N = N #No. of agents
#         T = T #Horizon
#         # y_state = cp.Variable((((T+1)*nx + T * nu, 1)))
#         Q = Q #Positive definite
#         R = R #Positive definite ..
#         Qf = Qf #Positive definite
#         radius = radius #Collision threshold radius
#         Ad = Ad #State matrix
#         Bd = Bd #Input matrix
#         MAX_ITER = MAX_ITER

"""Define constants"""
n_states = 6
n_inputs = 3
n_agents = 3
nx = n_states*n_agents
nu = n_inputs*n_agents
x0,xr = util.paper_setup_3_quads()
N = n_agents
T = 10
Q = sparse.diags([5., 5., 5., 1., 1., 1.]*N)
Qf = Q*500
R = 0.1*sparse.eye(N*n_inputs)
radius = 0.3
Ad, Bd = linear_kinodynamics(0.1, N)

cost = 0
# x_trj_var = cp.reshape(y_state[0:(T+1)*nx],[T+1, nx])
# u_trj_var = cp.reshape(y_state[(T+1)*nx:], [T, nu])
f_list = []
for id in range(N):
    for t in range(N):
        y_state = cp.Variable((((T+1)*nx + T * nu, 1)))
        cost += cp.quad_form(cp.reshape(y_state[0:(T+1)*nx],[T+1, nx])[t,:]-xr.flatten(),Q) + \
        cp.quad_form(cp.reshape(y_state[(T+1)*nx:], [T, nu])[t,:],R)

    cost += cp.quad_form((cp.reshape(y_state[0:(T+1)*nx],[T+1, nx])[-1,:]-xr.flatten()),Qf)    
    f_list.append(cost)


def run_worker(f, pipe):
    n_states = 6
    xbar = cp.Parameter((T+1)*nx + T*nu, value=np.zeros((T+1)*nx + T*nu))
    u = cp.Parameter((T+1)*nx + T*nu, value=np.zeros((T+1)*nx + T*nu)) 
    #This is the scaled Lagrange multiplier

    rho = 1
    f += (rho/2)*cp.sum_squares(y_state.flatten() - xbar + u)
    
    # ADMM loop
    # X_full = np.zeros((0,nx))
    # X_full = np.r_[X_full,x0.reshape(1,-1)]
    # U_full = np.zeros((0,nu))
    
    iter = 0
    while True:
        try:
            constr = []
            constr += [y_state[0:nx] == x0]
            for k in range(T):
                constr += [cp.reshape(y_state[:(T+1)*nx], [T+1, nx])[k+1,:] \
                            == Ad @ cp.reshape(y_state[:(T+1)*nx], [T+1, nx])[k] \
                                + Bd @ cp.reshape(y_state[(T+1)*nx:], [T, nu])[k]]  
                    
                # constr += [cp.reshape(y_state[(T+1)*nx:], [T, nu])[k]<= np.tile(np.array([3, 3, 3]),(N,))]
                # constr += [np.tile(np.array([-3, -3, -3]),(N,)) <= cp.reshape(y_state[(T+1)*nx:], [T, nu])[k]]
                
                #Solve local constrained problem with OSQP solver in CVXPY
                if iter > 0 :
                    pos_prev = state_prev[0]
                    # pos_prev = X_full[iter-1]
                    # pos_curr = cp.reshape(y_state[:(T+1)*nx],[T+1,nx])[k]
                    for i in range(N):
                        for j in range(N):
                            if j != i:
                                #See "Generation of collision-free trajectories for a quadrocopter fleet: 
                                # A sequential convex programming approach" for the linearization step;
                                constr += [cp.norm(pos_prev[j*n_states:j*n_states+3]-  \
                                pos_prev[i*n_states:i*n_states+3]) + \
                                (pos_prev[j*n_states:j*n_states+3].T- \
                                    pos_prev[i*n_states:i*n_states+3].T)/cp.norm(pos_prev[j*n_states:j*n_states+3] \
                                -pos_prev[i*n_states:i*n_states+3])@  \
                                ((cp.reshape(y_state[:(T+1)*nx],[T+1,nx])[k][j*n_states:j*n_states+3] \
                                -cp.reshape(y_state[:(T+1)*nx],[T+1,nx])[k][i*n_states:i*n_states+3])- \
                                (pos_prev[j*n_states:j*n_states+3]- \
                                    pos_prev[i*n_states:i*n_states+3]).flatten()) >= radius]


            prox = cp.Problem(cp.Minimize(f),constr)
            prox.solve(verbose=True)
            pipe.send(y_state.value.flatten())
            xbar.value = pipe.recv() #receive the averaged result from the main process.
            u.value += y_state.value.flatten() - xbar.value

            # ctrl = xbar.value[(T+1)*nx:].reshape((T,nu))[1]
            # xi = Ad @ xi + Bd @ ctrl
            # X_full = np.r_[X_full,xi]
            # U_full = np.r_[U_full, ctrl.reshape(1,-1)]
                    
            state_prev = xbar.value[0:(T+1)*nx]
            state_prev = state_prev.reshape((T+1, nx))

            iter += 1
            print(f'Current iteration is {iter}')
        except EOFError:
            print("Connection closed.")
            break
                
    

pipes = []
procs = []
for i in range(N):
    local, remote = Pipe()
    pipes += [local]
    procs += [Process(target=run_worker, args=(f_list[i], remote))]
    procs[-1].start()

MAX_ITER = 30
solution_list = []
for i in range(MAX_ITER):
    # Gather and average xi

    xbar = sum(pipe.recv() for pipe in pipes)/N
    # print(f'xbar is {xbar}, has shape {xbar.shape}\n')

    solution_list.append(xbar)
    # print(f'average of xbar is {np.mean(xbar)}\n')

    # Scatter xbar
    for pipe in pipes:
        pipe.send(xbar)

[p.terminate() for p in procs]
    


                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jun 19 04:47:59 PM: Your problem has 1152 variables, 11 constraints, and 576 parameters.
                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jun 19 04:47:59 PM: Your problem has 2016 variables, 11 constraints, and 576 parameters.
                                     CVXPY                                     
                                     v1.3.1                                    
===============================================================================(CVXPY) Jun 19 04:47:59 PM: It is compliant with the following grammars: DCP, DQCP

(CVXPY) Jun 19 04:47:59 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 19 04:47:59 PM: CVXPY will first compile your problem; then, it will

Process Process-15:
Traceback (most recent call last):
  File "/home/randychen233/anaconda3/envs/ICON_lab/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/randychen233/anaconda3/envs/ICON_lab/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_11912/3946925206.py", line 96, in run_worker
    prox.solve(verbose=True)
  File "/home/randychen233/anaconda3/envs/ICON_lab/lib/python3.10/site-packages/cvxpy/problems/problem.py", line 493, in solve
    return solve_func(self, *args, **kwargs)
  File "/home/randychen233/anaconda3/envs/ICON_lab/lib/python3.10/site-packages/cvxpy/problems/problem.py", line 1068, in _solve
    self.unpack_results(solution, solving_chain, inverse_data)
  File "/home/randychen233/anaconda3/envs/ICON_lab/lib/python3.10/site-packages/cvxpy/problems/problem.py", line 1393, in unpack_results
    raise error.SolverError(
cvxpy.error.SolverError: S


ECOS 2.0.10 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -5.157e-14  +2.049e+01  +1e+06  2e-01  2e-04  1e+00  2e+04    ---    ---    1  1  - |  -  - 
 1  +2.042e+02  +2.934e+02  +1e+06  9e-02  2e-04  7e+01  2e+04  0.0065  7e-01   1  2  1 |  0  0
 2  +5.178e+02  +6.978e+02  +1e+06  7e-02  2e-04  2e+02  2e+04  0.0074  8e-01   1  1  1 |  0  0
 3  +3.847e+02  +6.299e+02  +1e+06  6e-02  2e-04  2e+02  2e+04  0.0074  6e-01   1  1  1 |  0  0
 4  +3.569e+02  +6.295e+02  +1e+06  6e-02  2e-04  3e+02  2e+04  0.0021  8e-01   1  1  1 |  0  0
 5  +2.749e+02  +6.370e+02  +1e+06  5e-02  2e-04  3e+02  2e+04  0.0073  8e-01   1  1  1 |  0  0
 6  +2.061e+02  +6.407e+02  +1e+06  5e-02  2e-04  4e+02  2e+04  0.0050  9e-01   1  1  1 |  0  0
 7  +1.521e+02  +7.950e+02  +1e+06  3e-02  2e-04  6e+02  2e+04  0.0354  4e-01   1  1  1 |  0  0
 8  +6.453e+00  +8.553e+02  +1e+06  3e-02  2e

KeyboardInterrupt: 

### ADMM with only consensus constraints (toy example)

In [120]:
n_states = 6
n_inputs = 3
n_agents = 3
nx = n_states*n_agents
nu = n_inputs*n_agents
x0,xr = util.paper_setup_3_quads()
N = n_agents
T = 15
Q = sparse.diags([5., 5., 5., 1., 1., 1.]*N)
Qf = Q*500
R = 0.1*sparse.eye(N*n_inputs)



# x_trj_var = cp.reshape(y_state[0:(T+1)*nx],[T+1, nx])
# u_trj_var = cp.reshape(y_state[(T+1)*nx:], [T, nu])
f_list = []
for id in range(N):
    y_state = cp.Variable((((T+1)*nx + T * nu, 1)))
    cost = 0
    for t in range(T):
        #Quadratic tracking cost
        # cost += cp.quad_form(cp.reshape(y_state[:(T+1)*nx],[T+1, nx])[t,:]-xr.flatten(),Q) + \
        # cp.quad_form(cp.reshape(y_state[(T+1)*nx:], [T, nu])[t,:],R)
        cost += cp.quad_form(y_state[:(T+1)*nx][t*nx:(t+1)*nx]-xr, Q) + \
        cp.quad_form(y_state[(T+1)*nx:][t*nu:(t+1)*nu], R)
    cost += cp.quad_form(y_state[:(T+1)*nx][T*nx:(T+1)*nx]-xr, Qf)  
    f_list.append(cost)


def run_worker(f, pipe):

    xbar = cp.Parameter((T+1)*nx + T*nu, value=np.zeros((T+1)*nx + T*nu))
    u = cp.Parameter((T+1)*nx + T*nu, value=np.zeros((T+1)*nx + T*nu)) 
    #This is the scaled Lagrange multiplier

    rho = 1
    f += (rho/2)*cp.sum_squares(y_state.flatten() - xbar + u)
    
    # ADMM loop
    
    iter = 0
    prox = cp.Problem(cp.Minimize(f))
    while True:
        try:
            prox.solve(verbose=True)
            print(f'updated augmented state has value {y_state.value.flatten()}')
            pipe.send(y_state.value.flatten())
            xbar.value = pipe.recv() #receive the averaged result from the main process.
            u.value += y_state.value.flatten() - xbar.value

            iter += 1
            print(f'Current iteration is {iter}')
        except EOFError:
            print("Connection closed.")
            break
                
    

pipes = []
procs = []
for i in range(N):
    local, remote = Pipe()
    pipes += [local]
    procs += [Process(target=run_worker, args=(f_list[i], remote))]
    procs[-1].start()

MAX_ITER = 50
solution_list = []
for i in range(MAX_ITER):
    # Gather and average xi

    xbar = sum(pipe.recv() for pipe in pipes)/N
    # print(f'xbar is {xbar}, has shape {xbar.shape}\n')

    solution_list.append(xbar)
    # print(f'average of xbar is {np.mean(xbar)}\n')

    # Scatter xbar
    for pipe in pipes:
        pipe.send(xbar)

[p.terminate() for p in procs]

                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jun 20 10:27:39 AM: Your problem has 846 variables, 0 constraints, and 846 parameters.
                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jun 20 10:27:39 AM: Your problem has 846 variables, 0 constraints, and 846 parameters.
                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jun 20 10:27:39 AM: Your problem has 423 variables, 0 constraints, and 846 parameters.
(CVXPY) Jun 20 10:27:39 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 20 10:27:39 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 20 10:27:39

[None, None, None]

In [122]:
len(solution_list)

50

In [126]:
xr.T

array([[2.5, 1.5, 1. , 0. , 0. , 0. , 0.5, 1.5, 1. , 0. , 0. , 0. , 1.5,
        2.2, 1. , 0. , 0. , 0. ]])

In [125]:
solution_list[-1][:(T+1)*nx].reshape((T+1, nx))

array([[2.5, 1.5, 1. , 0. , 0. , 0. , 0.5, 1.5, 1. , 0. , 0. , 0. , 1.5,
        2.2, 1. , 0. , 0. , 0. ],
       [2.5, 1.5, 1. , 0. , 0. , 0. , 0.5, 1.5, 1. , 0. , 0. , 0. , 1.5,
        2.2, 1. , 0. , 0. , 0. ],
       [2.5, 1.5, 1. , 0. , 0. , 0. , 0.5, 1.5, 1. , 0. , 0. , 0. , 1.5,
        2.2, 1. , 0. , 0. , 0. ],
       [2.5, 1.5, 1. , 0. , 0. , 0. , 0.5, 1.5, 1. , 0. , 0. , 0. , 1.5,
        2.2, 1. , 0. , 0. , 0. ],
       [2.5, 1.5, 1. , 0. , 0. , 0. , 0.5, 1.5, 1. , 0. , 0. , 0. , 1.5,
        2.2, 1. , 0. , 0. , 0. ],
       [2.5, 1.5, 1. , 0. , 0. , 0. , 0.5, 1.5, 1. , 0. , 0. , 0. , 1.5,
        2.2, 1. , 0. , 0. , 0. ],
       [2.5, 1.5, 1. , 0. , 0. , 0. , 0.5, 1.5, 1. , 0. , 0. , 0. , 1.5,
        2.2, 1. , 0. , 0. , 0. ],
       [2.5, 1.5, 1. , 0. , 0. , 0. , 0.5, 1.5, 1. , 0. , 0. , 0. , 1.5,
        2.2, 1. , 0. , 0. , 0. ],
       [2.5, 1.5, 1. , 0. , 0. , 0. , 0.5, 1.5, 1. , 0. , 0. , 0. , 1.5,
        2.2, 1. , 0. , 0. , 0. ],
       [2.5, 1.5, 1. , 0. , 0. , 0. ,